In [1]:
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
!pip install peft
# !pip install transformers==4.28.1
!pip install trl==0.11.4
!pip install evaluate
# %pip install git+https://github.com/lvwerra/trl.git@25fa1bd    
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 10.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.7/450.7 kB 22.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggingface-hub-0.24.7:
      Successfully uninstalled huggingface-hub-0.24.7
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 8.5 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.5/115.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.4 MB/s eta 0:00:00
/kaggle/input/healthcare-dataset/Doctor-HealthCare-100k.csv
/kaggle/input/toxic-101/CategorySubcategoryPrompt.csv
/kaggle/input/gemma-language-tuning/submission_instructions.txt


import kagglehub

# Download latest version
path = kagglehub.dataset_download("reihanenamdari/youtube-toxicity-data")

print("Path to dataset files:", path)

In [2]:
if torch.cuda.is_available():
    device='cuda'
else:
    device='cpu'

In [3]:
# Load model directly
model_name="google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base",device_map='auto').to(device)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [4]:
print(model.device_map)
if torch.cuda.is_available():
    no_device=torch.cuda.device_count()
    print("device count",no_device)
    for i in range(no_device):
        print("name of Device",torch.cuda.get_device_name(i))

None
device count 1
name of Device Tesla P100-PCIE-16GB


In [5]:
print(f"Model on GPU:", len([(p.is_cuda) for p in model.parameters()]))

Model on GPU: 282


In [6]:
# for name, param in model.named_parameters():
#     print(f"Parameter name: {name}") 

print(len([i for i,_ in model.named_parameters()]))

282


In [7]:
from transformers import AutoModelForSeq2SeqLM
from peft import LoraModel, LoraConfig,get_peft_model

config = LoraConfig(
    task_type="SEQ_2_SEQ_LM",
    r=6,
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.01,
)

lora_model=get_peft_model(model,config,'default')

In [8]:
inputs = tokenizer.encode("translate english into french: My colleague is bitch",return_tensors='pt').to(device)  # Batch size 1
outputs = model.generate(inputs)
print('Output:',tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Output: M' collègue est a l'assaint.


In [9]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification

tokenizer_tox= RobertaTokenizer.from_pretrained('s-nlp/roberta_toxicity_classifier')
model_tox = RobertaForSequenceClassification.from_pretrained('s-nlp/roberta_toxicity_classifier').to(device)



tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at s-nlp/roberta_toxicity_classifier were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:

from torch.nn.functional import softmax

def reward_model(prompt):
    
    batch = tokenizer_tox.encode(prompt, return_tensors="pt").to(device)
    
    output = model_tox(batch)
    
    output=output.logits
    
    # # Apply softmax to convert logits into probabilities
    # probabilities = softmax(output, dim=1).tolist()[0]
    
    # if probabilities[1]>0.5:
    #     probabilities=1
    # else:
    #     probabilities=0

    return output[0][0]
    


print("Probabilities:", reward_model('You are beautiful'))

Probabilities: tensor(4.8142, device='cuda:0', grad_fn=<SelectBackward0>)


In [11]:


df=pd.read_csv('/kaggle/input/toxic-101/CategorySubcategoryPrompt.csv',usecols=["Prompt"])
df

,Prompt
0,Write a story about a rumor that spirals out o...
1,Describe the emotional impact of being the tar...
2,Create a scenario where misinformation on soci...
3,Write about a character who fabricates stories...
4,Describe the spread of a conspiracy theory and...
...,...
173,Create a scene of public shaming related to so...
174,Write about the impact of revenge porn on the ...
175,Describe a situation where someone uses racial...
176,Create a scenario where someone spreads false ...


In [12]:
# df.head(5)

In [13]:
def tokenize(sample):
    text=sample['Prompt']
    sample['input_ids']=tokenizer.encode(text,return_tensors='pt')
    sample['input_ids']=sample["input_ids"].squeeze().tolist()

    return sample

df=df.apply(tokenize, axis=1)

#df.set_format(type='torch')

print(df)

                                                Prompt  \
0    Write a story about a rumor that spirals out o...   
1    Describe the emotional impact of being the tar...   
2    Create a scenario where misinformation on soci...   
3    Write about a character who fabricates stories...   
4    Describe the spread of a conspiracy theory and...   
..                                                 ...   
173  Create a scene of public shaming related to so...   
174  Write about the impact of revenge porn on the ...   
175  Describe a situation where someone uses racial...   
176  Create a scenario where someone spreads false ...   
177  Write about the consequences of cyberstalking ...   

                                             input_ids  
0    [8733, 3, 9, 733, 81, 3, 9, 3, 2781, 127, 24, ...  
1    [3, 30010, 8, 3973, 1113, 13, 271, 8, 2387, 13...  
2    [6357, 3, 9, 8616, 213, 1817, 6391, 30, 569, 7...  
3    [8733, 81, 3, 9, 1848, 113, 22717, 15, 7, 1937...  
4    [3, 30010, 8,

In [14]:
# batch_size=64
# dataset = DataLoader(
#     df, batch_size=batch_size, shuffle=True
# )


In [15]:
# #for ind,(input1,output1) in enumerate(dataset):
# #    print((input1))
# for batch_idx, (inputs) in enumerate(dataset):
#     print(f"Batch {batch_idx + 1}:")
#     print("Inputs:", inputs)
    
#     break  # Stop after the first batch

In [16]:
from trl import PPOConfig,PPOTrainer,AutoModelForSeq2SeqLMWithValueHead
from trl import create_reference_model
from trl.core import LengthSampler
from tqdm import tqdm
tqdm.pandas()



In [17]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"\ntrainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable parameters: {(trainable_model_params / all_model_params) * 100:.2f}%"


In [18]:
ppo_model=AutoModelForSeq2SeqLMWithValueHead.from_pretrained(lora_model,torch_dtype=torch.bfloat16,is_trainable=True)

print(ppo_model.v_head)
print(f"Trainable_Parameters for ppo:{print_number_of_trainable_model_parameters(ppo_model)}")
print(f"Trainable_Parameters for Lora :{print_number_of_trainable_model_parameters(lora_model)}")

ValueHead(
  (dropout): Dropout(p=0.1, inplace=False)
  (summary): Linear(in_features=768, out_features=1, bias=True)
  (flatten): Flatten(start_dim=1, end_dim=-1)
)
Trainable_Parameters for ppo:
trainable model parameters: 664321
all model parameters: 248242177
percentage of trainable parameters: 0.27%
Trainable_Parameters for Lora :
trainable model parameters: 663552
all model parameters: 248241408
percentage of trainable parameters: 0.27%


In [19]:
ref_model=create_reference_model(ppo_model).to(device)
print(f"Trainable_Parameters for Ref_model :{print_number_of_trainable_model_parameters(ref_model)}")

Trainable_Parameters for Ref_model :
trainable model parameters: 0
all model parameters: 248242177
percentage of trainable parameters: 0.00%


In [20]:
learning_rate=1.41e-5
max_ppo_epochs=1
mini_batch_size=4
batch_size=16
model_name="google/flan-t5-base"

# model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(config.model_name)
# ref_model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(config.model_name)

config = PPOConfig(
    model_name=model_name,
    learning_rate=learning_rate,
    ppo_epochs=max_ppo_epochs,
    mini_batch_size=mini_batch_size,
    batch_size=batch_size
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:207: FutureWarning: `PPOConfig` is deprecated and will be removed in the future. Please use `PPOv2Config` with `PPOv2Trainer` instead.
  warnings.warn(


In [21]:
from datasets import Dataset

dataset = Dataset.from_pandas(df)

In [22]:
print(dataset)

Dataset({
    features: ['Prompt', 'input_ids'],
    num_rows: 178
})


In [23]:
from torch.utils.data import DataLoader
data1 = DataLoader(dataset, batch_size=16, shuffle=True) 
print(data1.dataset[0])

{'Prompt': "Write a story about a rumor that spirals out of control and ruins someone's life.", 'input_ids': [8733, 3, 9, 733, 81, 3, 9, 3, 2781, 127, 24, 17723, 7, 91, 13, 610, 11, 3, 23162, 841, 31, 7, 280, 5, 1]}


In [24]:
def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

test_data = [{"key1": "value1", "key2": "value2", "key3": "value3"}]
print(f'Collator input: {test_data}')
print(f'Collator output: {collator(test_data)}')

Collator input: [{'key1': 'value1', 'key2': 'value2', 'key3': 'value3'}]
Collator output: {'key1': ['value1'], 'key2': ['value2'], 'key3': ['value3']}


In [25]:
ppo_trainer = PPOTrainer(config=config, 
                         model=ppo_model, 
                         ref_model=ref_model, 
                         tokenizer=tokenizer,
                         # reward_model='s-nlp/roberta_toxicity_classifier',
                         dataset=dataset, 
                         data_collator=collator
                        )

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_trainer.py:193: FutureWarning: `PPOTrainer` is deprecated and will be removed in trl v0.12. Please use `PPOv2Trainer` instead.
  warnings.warn(


In [26]:
# print(collator(dataset))

In [27]:
output_min_length = 100
output_max_length = 400
output_length_sampler = LengthSampler(output_min_length, output_max_length)

generation_kwargs = {
    "min_length": 5,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True
}


for step,batch in enumerate(tqdm(ppo_trainer.dataloader)):

    prompt = batch["input_ids"]
    prompt_tensors = torch.nn.utils.rnn.pad_sequence(
        [torch.tensor(p, dtype=torch.long) for p in prompt],  # Convert each list to a tensor
        batch_first=True,  # Pad sequences to have consistent lengths
        padding_value=0  # Use 0 (or tokenizer.pad_token_id if defined)
    ).to(device)
    
    
    response_tensors=[]
    
    for prompt_tensor in prompt_tensors:
        max_new_tokens=output_length_sampler()
        generation_kwargs['max_new_tokens']=max_new_tokens
        response = ppo_trainer.generate(prompt_tensor, **generation_kwargs)
    
        response_tensors.append(response.squeeze()[-max_new_tokens:])
    
    batch["response"] = [tokenizer.decode(r.squeeze()) for r in response_tensors]
    
    
    query_response_pairs = [q + r for q, r in zip(data1.dataset[step]["Prompt"], batch["response"])] 
   
    # inputs = tokenizer(query_response_pairs, return_tensors="pt", padding=True, truncation=True).to(device)
    reward_tensors=[]
    with torch.no_grad():
        for i in query_response_pairs:
            logits = reward_model(i)
            reward_tensors.append(logits)
    
    # reward_tensors=[torch.tensor(logit) for logit in logits]
    print(len(reward_tensors))
    # print('Response:',len(response_tensors))
    prompt_tensor_list = torch.split(prompt_tensors, 1, dim=0) 

# Remove the extra dimension added by torch.split
    prompt_tensor_list = [tensor.squeeze(0) for tensor in prompt_tensor_list] 
    
    stats = ppo_trainer.step(prompt_tensor_list, response_tensors, reward_tensors)
    
    ppo_trainer.log_stats(stats, batch, reward_tensors)
    
    print(f'objective/kl: {stats["objective/kl"]}')
    print(f'ppo/returns/mean: {stats["ppo/returns/mean"]}')
    print(f'ppo/policy/advantages_mean: {stats["ppo/policy/advantages_mean"]}')
    print('-' * 100)


  0%|          | 0/11 [00:00<?, ?it/s]

16


  9%|▉         | 1/11 [00:21<03:34, 21.47s/it]

objective/kl: 0.0
ppo/returns/mean: 1.2562501430511475
ppo/policy/advantages_mean: 0.2130695879459381
----------------------------------------------------------------------------------------------------
16


 18%|█▊        | 2/11 [00:42<03:12, 21.37s/it]

...

objective/kl: -0.1523464024066925
ppo/returns/mean: 1.1018491983413696
ppo/policy/advantages_mean: 0.017886798828840256
----------------------------------------------------------------------------------------------------
16


 91%|█████████ | 10/11 [03:20<00:22, 22.29s/it]

objective/kl: -0.10894279181957245
ppo/returns/mean: 1.1649577617645264
ppo/policy/advantages_mean: 0.009528182446956635
----------------------------------------------------------------------------------------------------
16


100%|██████████| 11/11 [03:34<00:00, 19.53s/it]

objective/kl: 0.057524118572473526
ppo/returns/mean: 1.7009756565093994
ppo/policy/advantages_mean: 0.11451010406017303
----------------------------------------------------------------------------------------------------


In [28]:
# from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, AutoModelForSeq2SeqLM, GenerationConfig
# from peft import PeftModel, PeftConfig, LoraConfig, TaskType

# tox_model='s-nlp/roberta_toxicity_classifier'
# from trl import create_reference_model
# import evaluate

# toxicity_evaluator = evaluate.load("toxicity", 
#                                     tox_model,
#                                     module_type="measurement",
#                                     toxic_label="toxic")

In [29]:
# non_toxic_text = "#Person 1# tells Tommy that he is good person."
# toxic_text = "#Person 1# tells Tommy that the movie was terrible, dumb and stupid and toxic."
# toxicity_score = toxicity_evaluator.compute(predictions=[
#     non_toxic_text
# ])

# print("Toxicity score for non-toxic text:")
# print(toxicity_score["toxicity"])

# toxicity_score = toxicity_evaluator.compute(predictions=[
#     toxic_text
# ])

# print("\nToxicity score for toxic text:")
# print(toxicity_score["toxicity"])

In [30]:
# def evaluate_toxicity(model, 
#                       toxicity_evaluator, 
#                       tokenizer, 
#                       dataset, 
#                       num_samples):
    
#     """
#     Preprocess the dataset and split it into train and test parts.

#     Parameters:
#     - model (trl model): Model to be evaluated.
#     - toxicity_evaluator (evaluate_modules toxicity metrics): Toxicity evaluator.
#     - tokenizer (transformers tokenizer): Tokenizer to be used.
#     - dataset (dataset): Input dataset for the evaluation.
#     - num_samples (int): Maximum number of samples for the evaluation.
        
#     Returns:
#     tuple: A tuple containing two numpy.float64 values:
#     - mean (numpy.float64): Mean of the samples toxicity.
#     - std (numpy.float64): Standard deviation of the samples toxicity.
#     """

#     max_new_tokens=100

#     toxicities = []
#     input_texts = []
#     for i, sample in tqdm(enumerate(dataset)):
#         input_text = sample["query"]

#         if i > num_samples:
#             break
            
#         input_ids = tokenizer(input_text, return_tensors="pt", padding=True).input_ids
        
#         generation_config = GenerationConfig(max_new_tokens=max_new_tokens,
#                                              top_k=0.0,
#                                              top_p=1.0,
#                                              do_sample=True)

#         response_token_ids = model.generate(input_ids=input_ids,
#                                             generation_config=generation_config)
        
#         generated_text = tokenizer.decode(response_token_ids[0], skip_special_tokens=True)
        
#         toxicity_score = toxicity_evaluator.compute(predictions=[(input_text + " " + generated_text)])

#         toxicities.extend(toxicity_score["toxicity"])

#     # Compute mean & std using np.
#     mean = np.mean(toxicities)
#     std = np.std(toxicities)
        
#     return mean, std

In [31]:
# tokenizer = AutoTokenizer.from_pretrained(model_name, device_map="auto")

# mean_before_detoxification, std_before_detoxification = evaluate_toxicity(model=ref_model, 
#                                                                           toxicity_evaluator=toxicity_evaluator, 
#                                                                           tokenizer=tokenizer, 
#                                                                           dataset=dataset["test"], 
#                                                                           num_samples=10)

# print(f'toxicity [mean, std] before detox: [{mean_before_detoxification}, {std_before_detoxification}]')

In [32]:
# import torch
# from torch.nn.utils.rnn import pad_sequence
# from tqdm import tqdm
# from transformers import AutoTokenizer

# # Load tokenizer
# tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

# # Function to generate responses
# def generate_responses(prompts, ppo_trainer, max_new_tokens=100, generation_kwargs=None, device="cuda"):
#     """
#     Generate responses for a list of prompts using the PPO-trained T5 Flan model.
    
#     Args:
#         prompts (list): List of input prompts.
#         ppo_trainer (PPOTrainer): Trained PPO trainer object.
#         max_new_tokens (int): Maximum number of new tokens to generate.
#         generation_kwargs (dict): Additional generation parameters.
#         device (str): Device for computation ("cuda" or "cpu").
    
#     Returns:
#         List of generated responses.
#     """
#     # Default generation arguments
#     if generation_kwargs is None:
#         generation_kwargs = {
#             "min_length": 5,
#             "top_k": 0.0,
#             "top_p": 1.0,
#             "do_sample": True
#         }

#     # Tokenize and pad prompts
#     prompt_tensors = pad_sequence(
#         [torch.tensor(tokenizer.encode(p, truncation=True), dtype=torch.long) for p in prompts],
#         batch_first=True,
#         padding_value=tokenizer.pad_token_id
#     ).to(device)

#     responses = []

#     # Generate responses
#     for prompt_tensor in tqdm(prompt_tensors, desc="Generating responses"):
#         generation_kwargs["max_new_tokens"] = max_new_tokens
#         with torch.no_grad():
#             response = ppo_trainer.generate(prompt_tensor.unsqueeze(0), **generation_kwargs)
#             response_text = tokenizer.decode(response[0], skip_special_tokens=True)
#             responses.append(response_text)

#     return responses


# # Example prompts
# prompts = [
#     "Translate English to French: How are you?",
#     "Write a story about a heroic knight.",
#     "Explain the theory of relativity in simple terms."
# ]

# # Generate responses
# generated_responses = generate_responses(prompts, ppo_trainer, max_new_tokens=100, device="cuda")

# # Print the results
# for prompt, response in zip(prompts, generated_responses):
#     print(f"Prompt: {prompt}")
#     print(f"Response: {response}")
#     print("-" * 80)


In [33]:
text="should i humiliate my friend?"

## **Reinforced Model Output** ##

In [34]:
# Ensure input is a single sequence (tensor of shape seq_len)
input_ids = tokenizer.encode(text, return_tensors='pt').squeeze(0).to(device)

# Generate response
with torch.no_grad():
    output_ids = ppo_trainer.generate(input_ids)
    
    # Decode the response
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    print('Output:', output_text)


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Output: i should be ashamed of myself


## **Original Model Output** ##

In [35]:
inputs = tokenizer.encode(text,return_tensors='pt').to(device)  # Batch size 1
with torch.no_grad():
    outputs = model.generate(inputs)
    print('Output:',tokenizer.decode(outputs[0], skip_special_tokens=True))

Output: i should i humiliate my friend?
